## Importing the Data

In [1]:
# Constants
DATASET_DIR = './data/'
GLOVE_DIR = './glove.6B/'
SAVE_DIR = './'

import os
import pandas as pd

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [2]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


Minimum and Maximum Scores for each essay set.

In [3]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

## Preprocessing the Data

We will preprocess all essays and convert them to feature vectors so that they can be fed into the RNN.

These are all helper functions used to clean the essays.

In [4]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

## Defining the model 

Here we define a 2-Layer LSTM Model. 

Note that instead of using sigmoid activation in the output layer we will use
Relu since we are not normalising training labels.

In [5]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

Using TensorFlow backend.


## Training Phase

Now we train the model on the dataset.

We will use 5-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold.
We will then calculate Average Kappa for all the folds.

In [7]:
from sklearn.cross_validation import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

cv = KFold(len(X), n_folds=5, shuffle=True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv:
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training essays.
            sentences += essay_to_sentences(essay, remove_stopwords = True)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 8 models.
    if count == 5:
         lstm_model.save('./model_weights/final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1
        


--------Fold 1--------

Training Word2Vec Model...


/Users/ronitmankad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1, 300)            721200    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10380/10380 [==============================] - 6s 601us/step - loss: 63.5235 - mean_absolute_error: 4.3085
Epoch 2/50
10380/10380 [==============================] - 4s 402us/step - loss: 38.5016 - mean_absolute_error: 3.4547
Epoch 3/50
10380/10380 [========

10381/10381 [==============================] - 5s 438us/step - loss: 29.4525 - mean_absolute_error: 3.3367
Epoch 5/50
10381/10381 [==============================] - 5s 441us/step - loss: 27.6676 - mean_absolute_error: 3.2650
Epoch 6/50
10381/10381 [==============================] - 5s 451us/step - loss: 26.7322 - mean_absolute_error: 3.1840
Epoch 7/50
10381/10381 [==============================] - 5s 447us/step - loss: 25.1847 - mean_absolute_error: 3.0235
Epoch 8/50
10381/10381 [==============================] - 5s 451us/step - loss: 24.4046 - mean_absolute_error: 2.8881
Epoch 9/50
10381/10381 [==============================] - 5s 444us/step - loss: 22.0171 - mean_absolute_error: 2.7358
Epoch 10/50
10381/10381 [==============================] - 5s 447us/step - loss: 19.3726 - mean_absolute_error: 2.5699
Epoch 11/50
10381/10381 [==============================] - 5s 443us/step - loss: 16.9303 - mean_absolute_error: 2.3903
Epoch 12/50
10381/10381 [==============================] - 5s 448

10381/10381 [==============================] - 4s 432us/step - loss: 13.7165 - mean_absolute_error: 2.1590
Epoch 16/50
10381/10381 [==============================] - 6s 547us/step - loss: 13.3761 - mean_absolute_error: 2.0890
Epoch 17/50
10381/10381 [==============================] - 5s 484us/step - loss: 12.6187 - mean_absolute_error: 2.0184
Epoch 18/50
10381/10381 [==============================] - 5s 469us/step - loss: 12.5590 - mean_absolute_error: 2.0113
Epoch 19/50
10381/10381 [==============================] - 5s 468us/step - loss: 11.9494 - mean_absolute_error: 1.9737
Epoch 20/50
10381/10381 [==============================] - 5s 454us/step - loss: 11.9072 - mean_absolute_error: 1.9554
Epoch 21/50
10381/10381 [==============================] - 4s 419us/step - loss: 11.8927 - mean_absolute_error: 1.9263
Epoch 22/50
10381/10381 [==============================] - 5s 448us/step - loss: 11.4953 - mean_absolute_error: 1.8964
Epoch 23/50
10381/10381 [==============================] - 5

10381/10381 [==============================] - 4s 430us/step - loss: 10.9807 - mean_absolute_error: 1.8435
Epoch 27/50
10381/10381 [==============================] - 4s 421us/step - loss: 10.9244 - mean_absolute_error: 1.8338
Epoch 28/50
10381/10381 [==============================] - 4s 400us/step - loss: 10.6652 - mean_absolute_error: 1.7977
Epoch 29/50
10381/10381 [==============================] - 4s 419us/step - loss: 10.0984 - mean_absolute_error: 1.7716
Epoch 30/50
10381/10381 [==============================] - 4s 401us/step - loss: 10.0962 - mean_absolute_error: 1.7613
Epoch 31/50
10381/10381 [==============================] - 4s 423us/step - loss: 10.0946 - mean_absolute_error: 1.7547
Epoch 32/50
10381/10381 [==============================] - 4s 407us/step - loss: 9.8874 - mean_absolute_error: 1.7351
Epoch 33/50
10381/10381 [==============================] - 6s 550us/step - loss: 9.7172 - mean_absolute_error: 1.7241
Epoch 34/50
10381/10381 [==============================] - 7s 

10381/10381 [==============================] - 5s 482us/step - loss: 9.1404 - mean_absolute_error: 1.6770 2s - lo
Epoch 38/50
10381/10381 [==============================] - 5s 482us/step - loss: 9.1458 - mean_absolute_error: 1.6917
Epoch 39/50
10381/10381 [==============================] - 5s 477us/step - loss: 8.8663 - mean_absolute_error: 1.6622
Epoch 40/50
10381/10381 [==============================] - 4s 429us/step - loss: 9.2812 - mean_absolute_error: 1.6708
Epoch 41/50
10381/10381 [==============================] - 4s 427us/step - loss: 8.7367 - mean_absolute_error: 1.6541
Epoch 42/50
10381/10381 [==============================] - 4s 423us/step - loss: 9.1140 - mean_absolute_error: 1.6701
Epoch 43/50
10381/10381 [==============================] - 6s 577us/step - loss: 8.7978 - mean_absolute_error: 1.6674
Epoch 44/50
10381/10381 [==============================] - 5s 481us/step - loss: 8.5478 - mean_absolute_error: 1.6401
Epoch 45/50
10381/10381 [==============================] - 5

The Avg. Kappa Score is 0.961 which is the highest we have ever seen on this dataset.

In [10]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.9613
